In [1]:
# part 1

import os, re, sys
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, operations = s.split("\n\n")


calculated = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}


operations = operations.split("\n")
dependencies = []
calculations = []
for operation in operations:
    args, outcome = operation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    calculations.append((args[1], outcome))



def calculate(deps, calcs):

    operation, res = calcs
    a1, a2 = deps
    if operation == 'OR':
        calculated[res] = calculated[a1] | calculated[a2]
    elif operation == 'XOR':
        calculated[res] = calculated[a1] ^ calculated[a2]
    elif operation == 'AND':
        calculated[res] = calculated[a1] & calculated[a2]


to_do = list(range(len(dependencies)))
while to_do:
    #print(to_do)
    for i in to_do:
        deps = dependencies[i]
        if deps[0] in calculated and deps[1] in calculated:
            calculate(deps, calculations[i])
            to_do.remove(i)
            break



count, total = 1, 0
for i in range(0, 100):
    val = 'z' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        break
    #result += str(int(calculated[val]))
    total += count * int(calculated[val])
    count *= 2

print(total)


59619940979346
